# Grab Altmetric.com Data

## Request Functions
This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [19]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math


In [20]:
origin_folder = 'data/outputs/OR'

## Grab altmetric.com Ids

In [21]:
def get_altmetric_id(doi):
    detail_id = ''
    res = grab_from_url_json('https://api.altmetric.com/v1/doi/' + doi)
    if res is not None:
        detail_id = res['altmetric_id']
    return detail_id

In [22]:
all_articles_df = pd.read_excel(f'{origin_folder}/all.xlsx', usecols=['SO', 'DI'])
df = pd.DataFrame(columns=['DI', 'altmetric_id'])

CPU times: user 3.37 s, sys: 59.8 ms, total: 3.43 s
Wall time: 3.56 s


In [50]:
%%time
for index, doi in enumerate(all_articles_df['DI']):
    if doi in df['DI'].values:
        continue
    if index % 100 == 0:
        print(index)
    dfx = pd.DataFrame(columns=['DI', 'altmetric_id'])
    dfx['DI'] = [doi]
    dfx['altmetric_id'] = [get_altmetric_id(str(doi))]
    df = df.append(dfx, ignore_index=True)

df.to_csv(f'{origin_folder}/all_altmetric_id.csv', index=False)

/Users/mikezhu/Dev/miniconda2/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
2500
3400
xxxxxxxHTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries exceeded with url: /v1/doi/nan (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x123600940>: Failed to establish a new connection: [Errno 60] Operation timed out'))
xxxxxxxHTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries exceeded with url: /v1/doi/nan (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1237fd710>: Failed to establish a new connection: [Errno 60] Operation timed out'))
xxxxxxxHTTPSConnectionPool(host='api.altmetric.com', port=443): Max retries 

## Grab altmetric.com Details

In [58]:
def grab_detail_altmetric(doi, altmetric_id):
    df = pd.DataFrame()
    b_list = ['news outlets', 'blogs', 'policy', 'tweeters', 'weibo', 'facebook pages', 'wikipedia', 'redditors', 'f1000', 'video uploader', 'dimensions_citation', 'mendeley', 'citeulike']

    df['DI'] = [doi]
    df['altmetric_id'] = [altmetric_id]
    if altmetric_id != '' and not math.isnan(altmetric_id):
        altmetric_id = int(altmetric_id)
        news_anchor = 'news</dt><dd><a href="/details/' + str(altmetric_id) + '/news"><strong>'
        blogs_anchor = 'blogs</dt><dd><a href="/details/' + str(altmetric_id) + '/blogs"><strong>'
        policy_anchor = 'policy</dt><dd><a href="/details/' + str(altmetric_id) + '/policy-documents"><strong>'
        twitter_anchor = 'twitter</dt><dd><a href="/details/' + str(altmetric_id) + '/twitter"><strong>'
        weibo_anchor = 'weibo</dt><dd><a href="/details/' + str(altmetric_id) + '/weibo"><strong>'
        facebook_anchor = 'facebook</dt><dd><a href="/details/' + str(altmetric_id) + '/facebook"><strong>'
        wikipedia_anchor = 'wikipedia</dt><dd><a href="/details/' + str(altmetric_id) + '/wikipedia"><strong>'
        redditors_anchor = 'reddit</dt><dd><a href="/details/' + str(altmetric_id) + '/reddit"><strong>'
        f1000_anchor = 'f1000</dt><dd><a href="/details/' + str(altmetric_id) + '/f1000"><strong>'
        video_anchor = 'video</dt><dd><a href="/details/' + str(altmetric_id) + '/video"><strong>'
        dimensions_citation_anchor = 'dimensions_citation</dt><dd><a href="/details/' + str(altmetric_id) + '/citations"><strong>'
        mendeley_anchor = 'mendeley</dt><dd><a href="/details/' + str(altmetric_id) + '#mendeley-demographics"><strong>'
        citeulike_anchor = 'citeulike</dt><dd><strong>'
        c_list = [news_anchor, blogs_anchor, policy_anchor, twitter_anchor, weibo_anchor, facebook_anchor, wikipedia_anchor, redditors_anchor, f1000_anchor, video_anchor, dimensions_citation_anchor, mendeley_anchor, citeulike_anchor]

        end_anchor = '</strong>'

        res = grab_from_url_content('https://www.altmetric.com/details/' + str(altmetric_id))
        if res is not None:

            for i in range(0, len(c_list)):
                start_index = res.find(c_list[i])
                if start_index > 0:
                    start_index += len(c_list[i])
                    number = 0
                    end_index = res.find(end_anchor, start_index, start_index + 100)
                    number_temp = res[start_index: end_index]

                    if number_temp is not '':
                        number = number_temp
                    df[b_list[i]] = int(number)
                else:
                    df[b_list[i]] = 0
        else:
            for i in range(0, len(b_list)):
                df[b_list[i]] = 0
    else:
        for i in range(0, len(b_list)):
            df[b_list[i]] = 0

    return df

In [60]:
%%time
all_articles_df = pd.read_csv(f'{origin_folder}/all_altmetric_id.csv')
df = pd.DataFrame(columns=['DI'])
for index, row in all_articles_df.iterrows():
    if row['DI'] in df['DI'].values:
        continue
    if index % 100 == 0:
        print(index)
    dfx = grab_detail_altmetric(row['DI'], row['altmetric_id'])
    df = df.append(dfx, ignore_index=True)

df.to_csv(f'{origin_folder}/all_altmetric_detail.csv', index=False)

0
/Users/mikezhu/Dev/miniconda2/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
CPU times: user 1min 29s, sys: 2.61 s, total: 1min 32s
Wall time: 38min 24s


## Grab altmetric.com's Tweets Detail

### Parser
We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [61]:
from html.parser import HTMLParser

class AltmetricHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_reply = False
    has_article = False

    def handle_starttag(self, tag, attrs):
        if tag == 'body':
            self.has_article = False
        if tag == 'article':
            self.has_article = True
            self.in_article = True
        
        if self.in_article and (tag == 'a'):
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'reply'):
                    self.is_reply = True
                if self.is_reply and (attr[0] == 'href'):
                    self.articles.append(attr[1].split('=')[1])
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_reply = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = AltmetricHTMLParser()

In [62]:
redirect_str = '<html><body>You are being <a href="https://www.altmetric.com/details/4236878">redirected</a>.</body></html>'

### Grab Data

In [71]:
import pandas as pd
import numpy as np

df = pd.read_csv(f'{origin_folder}/all_altmetric_id.csv', usecols=['DI', 'altmetric_id'], dtype={'altmetric_id':str})
df

,DI,altmetric_id
0,10.1007/s10479-015-1981-7,NaN
1,10.1007/s10479-015-1987-1,NaN
2,10.1007/s10479-015-1880-y,NaN
3,10.1007/s10479-015-1958-6,NaN
4,10.1007/s10479-015-1935-0,NaN
5,10.1007/s10479-015-1905-6,NaN
6,10.1007/s10479-015-1997-z,NaN
7,10.1007/s10479-015-2010-6,NaN
8,10.1007/s10479-015-1913-6,NaN
9,10.1007/s10479-015-1925-2,NaN


In [75]:
headers = {
    'cookie': "_ga=GA1.2.501259425.1572141776; weibo_license_acknowledgement=false; explorer_user=WWs2eDNOOUVjVHJTcHRjSm9Oc0ZuZEh0eDRkTmlPd0tCWFMzODNyNlIweFBvdjllM3RkcmF0RVdtRWdvRDN4Ny0tQlNpeTBsWU5NK0ZNTnFpK1FpVFUzdz09--8bac413ca9fd623b9918b311729130cfbb4e66e1; _altmetric-explorer_session=eExKU3g1UmJhWXpXbmVtS05YZmxVS3YyaVNDSGdPL2JyaWVYaE9mRTdPT3RlYlRLaTdrRmRWcnZpeFMrMFJLb0FIVDBvNHI3N09MWS9HZDhDRDdIbTZPZElvMWNPNzMvTm5JVkovWThiQVplZVdTRlZheDBWYWdvd2lIMXo2NHJUZkFJd2MxcWpZSmpCaXVXRW9CYlZnZ1l4L1lXMGhIa1k1d0o1bllvS09uWDBQb3JXMWUrazZ2YWVwaUwzei93cDdUZkhwanFtSy9lSkxLUmdRdEIrZz09LS1DckIxZFJZUVBWNmhsTEJXcWtkR2x3PT0%3D--884ad24217914b545556a68678c2d803c65ce9cf; intercom-session-9dnltu6y=alcwSlZxZHMzZlY3Wm8yaFpvWFg1OGxEYVZ4MXRRUDMrbkk1Q01Bdm5XNXJkQXNZcVAyeEZYVHFYS3RUbWJSbi0tclFPUDlZcUVpMFZKQVlxdlMwRllVQT09--46bb4ebcf64063d6a3712dd82a1d2bff57eed87f, _ga=GA1.2.501259425.1572141776; weibo_license_acknowledgement=false; explorer_user=WWs2eDNOOUVjVHJTcHRjSm9Oc0ZuZEh0eDRkTmlPd0tCWFMzODNyNlIweFBvdjllM3RkcmF0RVdtRWdvRDN4Ny0tQlNpeTBsWU5NK0ZNTnFpK1FpVFUzdz09--8bac413ca9fd623b9918b311729130cfbb4e66e1; _altmetric-explorer_session=eExKU3g1UmJhWXpXbmVtS05YZmxVS3YyaVNDSGdPL2JyaWVYaE9mRTdPT3RlYlRLaTdrRmRWcnZpeFMrMFJLb0FIVDBvNHI3N09MWS9HZDhDRDdIbTZPZElvMWNPNzMvTm5JVkovWThiQVplZVdTRlZheDBWYWdvd2lIMXo2NHJUZkFJd2MxcWpZSmpCaXVXRW9CYlZnZ1l4L1lXMGhIa1k1d0o1bllvS09uWDBQb3JXMWUrazZ2YWVwaUwzei93cDdUZkhwanFtSy9lSkxLUmdRdEIrZz09LS1DckIxZFJZUVBWNmhsTEJXcWtkR2x3PT0%3D--884ad24217914b545556a68678c2d803c65ce9cf; intercom-session-9dnltu6y=alcwSlZxZHMzZlY3Wm8yaFpvWFg1OGxEYVZ4MXRRUDMrbkk1Q01Bdm5XNXJkQXNZcVAyeEZYVHFYS3RUbWJSbi0tclFPUDlZcUVpMFZKQVlxdlMwRllVQT09--46bb4ebcf64063d6a3712dd82a1d2bff57eed87f; Cookie_1=value",
    'User-Agent': "PostmanRuntime/7.19.0",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "30d13c93-aa9a-44e1-84e0-8876f98ae1ba,b45c8eb6-7f77-4c48-b961-921c89280079",
    'Host': "www.altmetric.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

In [76]:
%%time
articles_dict = {}
for index, row in df.iterrows():
    if row['altmetric_id'] == '' or str(row['altmetric_id']).lower() == 'nan':
        articles_dict[row['DI']] = {'altmetric_id': '', 'twitter_num': 0, 'tweets': []}
        continue
    # print(row['DI'], row['altmetric_id'])
    grab_url = f'https://www.altmetric.com/details/{row["altmetric_id"]}/twitter/page:'
    print(str(index), grab_url)
    parser.articles = []
    for i in range(1, 100000):
        parser.feed(grab_from_url_content(grab_url + str(i), headers=headers))
        print('page: ', str(i))
        if not parser.has_article:
            break
    articles_dict[row['DI']] = {'altmetric_id': row['altmetric_id'], 'twitter_num': len(parser.articles), 'tweets': parser.articles}

/www.altmetric.com/details/2533951/twitter/page:
page:  1
page:  2
5381 https://www.altmetric.com/details/2882514/twitter/page:
page:  1
page:  2
5383 https://www.altmetric.com/details/2964822/twitter/page:
page:  1
page:  2
5406 https://www.altmetric.com/details/3704930/twitter/page:
page:  1
page:  2
5411 https://www.altmetric.com/details/2757546/twitter/page:
page:  1
5417 https://www.altmetric.com/details/1276913/twitter/page:
page:  1
page:  2
5440 https://www.altmetric.com/details/1865485/twitter/page:
page:  1
page:  2
5469 https://www.altmetric.com/details/4847320/twitter/page:
page:  1
page:  2
5479 https://www.altmetric.com/details/14742308/twitter/page:
page:  1
5482 https://www.altmetric.com/details/4248723/twitter/page:
page:  1
page:  2
5484 https://www.altmetric.com/details/21369243/twitter/page:
page:  1
page:  2
5488 https://www.altmetric.com/details/35541458/twitter/page:
page:  1
5502 https://www.altmetric.com/details/64434130/twitter/page:
page:  1
page:  2
5508 htt

In [77]:
with open(f'{origin_folder}/article_tweets_altmetric.json', "w+") as dump_f:
    dump_f.write(json.dumps(articles_dict))